## Experimenting with Bedrock and Langchain for a RAG question-answering application

In [1]:
!pip install -U boto3 langchain==0.0.349 faiss-cpu transformers
!pip install amazon-textract-textractor pypdf Pillow 

**You will need an AWS account with access to the models tested bellow**

In [2]:
%env AWS_PROFILE=puzzle

env: AWS_PROFILE=puzzle


In [3]:
import boto3
import os

boto3.setup_default_session(profile_name='puzzle')

br = boto3.client('bedrock')
bedrock = boto3.client('bedrock-runtime')
print(os.getenv('AWS_PROFILE'))

puzzle


Loading some test documents

In [4]:
from langchain.docstore.document import Document

files = [f for f in os.listdir("./sample-data/")]
docs = []
for file in files:
    with open("./sample-data/" + file, 'rb') as f:
        docs.append(Document(page_content=f.read(), metadata={"source": "local", "title": file}))

docs

[Document(page_content='1. Features, Evaluation, and Treatment of Coronavirus (COVID-19).\r\n\r\nCascella M(1), Rajnik M, Aleem A(2), Dulebohn SC, Di Napoli R(3).\r\n\r\nIn: StatPearls [Internet]. Treasure Island (FL): StatPearls Publishing; 2023 \r\nJan–.\r\n2023 Aug 18.\r\n\r\nAuthor information:\r\n(1)Istituto Nazionale Tumori - IRCCS - Fondazione Pascale, Via Mariano Semmola \r\n80100, Napoli. Italy\r\n(2)Lehigh Valley Health Network, Allentown, USA\r\n(3)Institut Jules Bordet -Brussels, Belgium\r\n\r\nCoronavirus disease 2019 (COVID-19) is a highly contagious viral illness caused \r\nby severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). COVID-19 has \r\nhad a catastrophic effect on the world, resulting in more than 6 million deaths \r\nworldwide. After the first cases of this predominantly respiratory viral illness \r\nwere reported in Wuhan, Hubei Province, China, in late December 2019, SARS-CoV-2 \r\nrapidly disseminated worldwide. This compelled the World Health Orga

Creating an in-memory vector database

In [5]:
from langchain.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS

embeddings = BedrockEmbeddings(client=bedrock,model_id="amazon.titan-embed-text-v1")
vector_db = FAISS.from_documents(documents=docs, embedding=embeddings)

Testing the in-memory database and creating a retriever (wrapper on top to be used by langchain). MMR stads for 

**Maximal Marginal Relevance**

MMR considers the similarity of keywords/keyphrases with the document, along with the similarity of already selected keywords and keyphrases.

In [6]:
query = "What is diabetes?"
retriever = vector_db.as_retriever(search_type='mmr', search_kwargs={"k": 3})
relevant_docs = retriever.get_relevant_documents(query)   
relevant_docs

[Document(page_content='1. Diabetes.\r\n\r\nSapra A(1), Bhandari P(1).\r\n\r\nIn: StatPearls [Internet]. Treasure Island (FL): StatPearls Publishing; 2023 \r\nJan–.\r\n2023 Jun 21.\r\n\r\nAuthor information:\r\n(1)Southern Illinois University School of Medicine\r\n\r\nDiabetes mellitus is taken from the Greek word diabetes, meaning siphon - to \r\npass through and the Latin word\xa0mellitus\xa0meaning sweet. A review of the history \r\nshows that the term "diabetes" was first used by Apollonius of Memphis around \r\n250\xa0to 300 BC. Ancient Greek, Indian, and Egyptian civilizations discovered the \r\nsweet nature of urine in this condition, and hence the propagation of the word \r\nDiabetes Mellitus came into being. Mering and Minkowski, in 1889, discovered the \r\nrole of the pancreas in the pathogenesis of diabetes. In 1922 Banting, Best, and \r\nCollip purified the hormone insulin from the pancreas of cows at the University \r\nof Toronto, leading to the availability of an effectiv

Checking what text models does our account have access to

In [11]:
resp = br.list_foundation_models(
    byOutputModality='TEXT'
)
for model in resp['modelSummaries']:
    print(model['modelId'])

amazon.titan-text-express-v1:0:8k
amazon.titan-text-express-v1
anthropic.claude-instant-v1
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-v2:1
anthropic.claude-v2


Creating the PromptTemplate and the Bedrock client and the entire chain which connects the retriever to the LLM client. "stuff" for a type name is just amazing. You can read more about it here [What is chain_type='stuff'?](https://python.langchain.com/docs/modules/chains/document/stuff) 

In [8]:
from langchain.chains import RetrievalQA
from langchain.llms import Bedrock
from langchain.prompts import PromptTemplate

template = """

Answer the question as truthfully as possible strictly using only the provided text, and if the answer is not contained within the text, say "I don't know". Skip any preamble text and reasoning and give just the answer.

<text>{context}</text>
<question>{question}</question>
<answer>"""

qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])

chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }

bedrock_llm = Bedrock(client=bedrock, model_id="amazon.titan-text-express-v1")
qa = RetrievalQA.from_chain_type(
    llm=bedrock_llm, 
    chain_type="stuff", 
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)


### Now we test

In [9]:
question = "What is diabetes?"
result = qa.run(question)
print(result.strip())

A metabolic disease, involving inappropriately elevated blood glucose levels.</answer>


In [10]:
question = "How many deaths did Covid cause worldwide?"
result = qa.run(question)
print(result.strip())

6 million deaths worldwide</answer>


**Changing regions to test more models since in EU we have laws**

In [23]:
boto3.setup_default_session(profile_name='puzzle', region_name='us-east-1')

br = boto3.client('bedrock')

resp = br.list_foundation_models(
    byOutputModality='TEXT'
)
for model in resp['modelSummaries']:
    print(model['modelId'])

amazon.titan-tg1-large
amazon.titan-text-lite-v1:0:4k
amazon.titan-text-lite-v1
amazon.titan-text-express-v1:0:8k
amazon.titan-text-express-v1
ai21.j2-grande-instruct
ai21.j2-jumbo-instruct
ai21.j2-mid
ai21.j2-mid-v1
ai21.j2-ultra
ai21.j2-ultra-v1
anthropic.claude-instant-v1:2:100k
anthropic.claude-instant-v1
anthropic.claude-v1
anthropic.claude-v2:0:18k
anthropic.claude-v2:0:100k
anthropic.claude-v2:1:18k
anthropic.claude-v2:1:200k
anthropic.claude-v2:1
anthropic.claude-v2
cohere.command-text-v14:7:4k
cohere.command-text-v14
cohere.command-light-text-v14:7:4k
cohere.command-light-text-v14
meta.llama2-13b-chat-v1:0:4k
meta.llama2-13b-chat-v1
meta.llama2-70b-chat-v1:0:4k
meta.llama2-70b-chat-v1
meta.llama2-13b-v1:0:4k
meta.llama2-13b-v1
meta.llama2-70b-v1:0:4k
meta.llama2-70b-v1


**Testing the Llama 2 Chat 13 billion params from Meta**

In [30]:
from langchain.chains import RetrievalQA
from langchain.llms import Bedrock
from langchain.prompts import PromptTemplate

template = """

Answer the question as truthfully as possible strictly using only the provided text, and if the answer is not contained within the text, say "I don't know". Skip any preamble text and reasoning and give just the answer.

<text>{context}</text>
<question>{question}</question>
<answer>"""

bedrock = boto3.client('bedrock-runtime') # new client in US East 1

qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])

chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }

bedrock_llm = Bedrock(client=bedrock, model_id="meta.llama2-13b-chat-v1")
qa = RetrievalQA.from_chain_type(
    llm=bedrock_llm, 
    chain_type="stuff", 
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)

In [32]:
question = "What is diabetes?"
result = qa.run(question)
print(result.strip())

Diabetes mellitus is a metabolic disease involving inappropriately elevated blood glucose levels.</answer>


In [33]:
question = "How many deaths did Covid cause worldwide?"
result = qa.run(question)
print(result.strip())

6 million deaths worldwide</answer>


**Testing the Llama 2 Chat 70 billion params from Meta**

In [37]:
from langchain.chains import RetrievalQA
from langchain.llms import Bedrock
from langchain.prompts import PromptTemplate

template = """

Answer the question as truthfully as possible strictly using only the provided text, and if the answer is not contained within the text, say "I don't know". Skip any preamble text and reasoning and give just the answer.

<text>{context}</text>
<question>{question}</question>
<answer>"""

bedrock = boto3.client('bedrock-runtime') # new client in US East 1

qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])

chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }

bedrock_llm = Bedrock(client=bedrock, model_id="meta.llama2-70b-chat-v1")
qa = RetrievalQA.from_chain_type(
    llm=bedrock_llm, 
    chain_type="stuff", 
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)

In [38]:
question = "What is diabetes?"
result = qa.run(question)
print(result.strip())

Diabetes mellitus is a metabolic disease, involving inappropriately elevated blood glucose levels.</answer>


In [39]:
question = "How many deaths did Covid cause worldwide?"
result = qa.run(question)
print(result.strip())

Over 6 million deaths worldwide</answer>


AWS Bedrock Pricing (https://aws.amazon.com/bedrock/pricing/)